In [2]:
import pandas as pd
from sqlalchemy import create_engine, text

# 1. Create database connection
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
conn = engine.connect()

# 2. Define target date
target_date = '2025-05-26'

In [4]:
# 3. Extract data for the target date
sql_extract = f"SELECT * FROM price WHERE date = '{target_date}'"
temp_df = pd.read_sql(sql_extract, conn)
print(f"Initial temp DataFrame shape: {temp_df.shape}")

Initial temp DataFrame shape: (348, 7)


In [8]:
# 4. Delete records for target date from DB
# Check count before deletion
count_before = pd.read_sql(f"SELECT COUNT(*) FROM price WHERE date = '{target_date}'", conn).iloc[0,0]
try:
    sql_delete = text("DELETE FROM price WHERE date = :target_date")
    conn.execute(sql_delete, {"target_date": target_date})
    conn.commit()
    count_after = pd.read_sql(f"SELECT COUNT(*) FROM price WHERE date = '{target_date}'", conn).iloc[0,0]
    print(f"Deleted {count_before - count_after} records")
except Exception as e:
    conn.rollback()
    print(f"Error: {str(e)}")

Error: 'Connection' object has no attribute 'rowcount'


In [10]:
# 5. Deduplicate temp DataFrame
# Keep first occurrence of (name, date) combination
cleaned_df = temp_df.drop_duplicates(subset=['name', 'date'], keep='first')
print(f"Cleaned DataFrame shape: {cleaned_df.shape}")
print(f"Removed duplicates: {len(temp_df) - len(cleaned_df)}")

Cleaned DataFrame shape: (174, 7)
Removed duplicates: 174


In [12]:
# 6. Insert cleaned data back to database
cleaned_df.to_sql(
    name='price',
    con=engine,
    if_exists='append',
    index=False
)
print("Inserted cleaned data back to price table")

Inserted cleaned data back to price table


In [14]:
# 7. Verify final count
final_count = pd.read_sql("SELECT COUNT(*) FROM price", conn).iloc[0,0]
print(f"Final table count: {final_count}")

# Close connection
#conn.close()

Final table count: 358505
